In [ ]:
%load_ext autoreload
%autoreload 2

In [57]:
import sys
sys.path.append('../classes')
import numpy as np
import sympy as sym
from Material import Material
# from abdCalcs import ABDs
from SandwichPanel import TestSpecimen
from MomInertia import moments_of_inertia
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [58]:
Eglass6oz = Material("6oz_Eglass", 4305899, 4305899, 7.687E14, 0.17,  0.0903)
Eglass4oz = Material("4oz_Eglass", 4305899 * 2/3, 4305899 * 2/3,5.3E9*2/3, 0.17,  0.0903*2/3)
EPS_Foam = Material("EPS_Foam", 5511.56, 5511.56, 2755.78, 0.32, 0.00197)
PU_Foam = Material("PU_Foam", 31900.46, 31900.46, 0, 0, 0.00866)
Balsa = Material("Balsa", 319004.63, 319004.63, 15364.21, .25, 0.00903) 
Basswood = Material("Basswood", 1029013.39, 0, 0, 0.01156)
Carbon = Material("Carbon", 21760000000, 21760000000, 0, 0.0578)
w_balsa = 0.125
t = .25E-3*1/(0.0254)
h_eps = 2.5
w_eps = 10

In [59]:
construction = {'Stringer':[Balsa,w_balsa],'Top Lam':[[Eglass4oz,t],[Eglass6oz,t]],\
            'Core':[EPS_Foam,h_eps,w_eps],'Bot Lam':[[Eglass4oz,t]]}

TDid_101 = TestSpecimen(construction)


#### TODO

1. Implement graph into the class for the Test Specimen
       - Add max stress in the different parts of the test specimen.
2. Incorporate the moment and shear diagrams into the plot
3. Create parent class Test Specimen which gets the data, calls moment of inertia, and finds effective moduli of laminates
    - Then create sub classes based on the design (Vertical Stringer, Horizontal Stringer). These sub classes will have the ability to get the data and the EIyy etc, but they will have a different get_geometry method for their given shape. Use an abstract class. 
    - Fix the way that the geometry of the cross section is found. Use coordinates and not the stupid yzs thing.

In [125]:
#Define known values
P = -100 #pounds
Lw = 2  #inches
L = 10

# Define the symbols
a, b, c, x, = sym.symbols('a b c x')

# Define the function
w = a*x**2 + b*x + c

# Define the equations
eq1 = sym.Eq(sym.integrate(w, (x, -Lw, Lw)), P)
eq2 = sym.Eq(w.subs(x, -Lw), 0)
eq3 = sym.Eq(w.subs(x, Lw), 0)

# Solve the system of equations
solution = sym.solve((eq1, eq2, eq3), (a, b, c))
a_val = solution[a]
b_val = solution[b]
c_val = solution[c]

x_vals = np.linspace(-10, 10, 100)
fig = make_subplots(rows=3,cols=1)
#Plotting Distributed Load
w_solve = a_val*x**2 + b_val*x + c_val
wexpr1,wcond1 = 0, sym.And(-L<x,x<-Lw)
wexpr2,wcond2 = w_solve,sym.And(-Lw<x,x<Lw)
wexpr3,wcond3 = 0, sym.And(Lw<x,x<L)
w = sym.Piecewise((wexpr1,wcond1),(wexpr2,wcond2),(wexpr3,wcond3))
w_vals = np.array([float(sym.N(w.subs('x', x))) for x in x_vals])
w_val_at_0 = float(sym.N(w.subs('x', 0)))
fig.add_trace(go.Scatter(x=x_vals,y=w_vals,name='Distributed Load (lb/in)'),row=1,col=1)
fig.add_annotation(
    text=str(w_val_at_0)+" lbin", x=0, y=w_val_at_0, xref='x1', yref='y1', showarrow=False,
    font=dict(color='black'), bgcolor='white', bordercolor='black', borderwidth=1, borderpad=1
)

#Solving and Plotting Shear
Vexpr1,Vcond1 = -P/2, sym.And(-L<x,x<-Lw)
Vexpr2,Vcond2 = sym.integrate(w_solve,x),sym.And(-Lw<=x,x<=Lw)
Vexpr3,Vcond3 = P/2, sym.And(Lw<x,x<L)
V = sym.Piecewise((Vexpr1,Vcond1),(Vexpr2,Vcond2),(Vexpr3,Vcond3))
V_vals = np.array([float(sym.N(V.subs('x', x))) for x in x_vals])
V_val_at_minus_Lw = float(sym.N(V.subs('x', -Lw)))
fig.add_trace(go.Scatter(x=x_vals, y=V_vals,name='Distributed Shear (lb)'),row=2,col=1)
fig.add_annotation(
    text=str(V_val_at_minus_Lw)+" lb", x=-Lw, y=V_val_at_minus_Lw, xref='x2', yref='y2', showarrow=False,
    font=dict(color='black'), bgcolor='white', bordercolor='black', borderwidth=1, borderpad=1
)

#Solving for Moment
print(V.subs(x,-L)*L)
M = sym.integrate(V,x) - P/2*L
M_vals = np.array([float(sym.N(M.subs('x', x))) for x in x_vals])
M_val_at_0 = float(sym.N(M.subs('x', 0)))
fig.add_trace(go.Scatter(x=x_vals, y=M_vals,name='Distributed Moment (lb)'),row=3,col=1)
fig.add_annotation(
    text=str(M_val_at_0)+" lb*in", x=0.1, y=M_val_at_0, xref='x3', yref='y3', showarrow=False,
    font=dict(color='black'), bgcolor='white', bordercolor='black', borderwidth=1, borderpad=1
)
#Annotating the Plot
fig.update_yaxes(title_text="Distributed Load (lb/in)", row=1, col=1)
fig.update_yaxes(title_text="Distributed Shear (lb)", row=2, col=1)
fig.update_yaxes(title_text="Distributed Moment (lb)", row=3, col=1)
fig.update_xaxes(title_text="L (in)", row=3, col=1)
fig.update_layout(
    height=750,
    title={
        'text': "<b>Distributed Load, Shear, and Moment SURFURS</b>",
        'y':0.945,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'
    }
)
fig.add_annotation(
    text=f'Buckle Failure at: {-P} pounds',
    showarrow=False,
    xref='paper',
    yref='paper',
    x=0.5,  # x position in normalized coordinates (far right is 1)
    y=1.06,   # y position in normalized coordinates (top is 1)
    font=dict(
        size=16  # change the size to fit your needs
    )
)
fig.update_layout(height=750)
fig.show()


nan


In [ ]:
#Test is moments of Inertia works
E1 = 100
rho_1 = 2
E2 = 50
rho_2 = 5
top_lam = [[0,5,5,0],[4,4,6,6]] #Inches
string = [[2,3,3,2],[0,0,4,4]]
yzs = np.array([top_lam,string])
E = [E1,E2]
mats = mats = {'Top Lam':['orange',rho_1],'Bot Lam':['brown',rho_2]}
moments_of_inertia(yzs,E,mats)
